# Bonus-Track Assignment 3: Sequential MNIST classification task wih ESN

Solve the sequential MNIST classification problem with an ESN (see details on this task from previous lab assignment files).

## Libraries

In [1]:
import os
import copy

from torch import cuda, Tensor
from typing import Tuple
from itertools import product

# To work with Google colab
#!wget https://raw.githubusercontent.com/jacons/Computational-NeuroScience-Lab/master/Utils/utils.py
#!wget https://raw.githubusercontent.com/jacons/Computational-NeuroScience-Lab/master/LAB3_2/Assignment3/TorchEchoStateNetworks.py

from Utils.utils import Sequential_mnist
from LAB3_2.Assignment3.TorchEchoStateNetworks import sMNISTEsnClassifier

In [2]:
gpu = 'cuda' if cuda.is_available() else 'cpu'

## Grid-search function

In [3]:
class GridSearch:

    def __init__(self,parameters_grid:dict, tr:Tuple[Tensor,Tensor], dev:Tuple[Tensor,Tensor]):

        all_configs = [dict(zip(parameters_grid.keys(), configs)) for configs in product(*parameters_grid.values())]

        print("Number of configurations to try: ",len(all_configs))

        """
        Returns the performance in each configuration:

            rank = a list of results for each configuration
            best = best model used to final retrain
            loss = training loss history of the best model
        """
        rank, best = self.run(tr, dev, all_configs)

        # we sort by validation loss
        rank = sorted(rank, key=lambda conf: -conf[2])

        print("\nThe best solution in ", rank[0])
        self.best_config = rank[0][0]
        self.best_model = best

    @staticmethod
    def run(tr:Tuple[Tensor,Tensor], dev:Tuple[Tensor,Tensor], configs:list):
        """
        In the grid search, we explore all configurations provided and try to find the best
        hyperparameter configuration using the training set to train the model and the validation
        set to compare the performance among all models instantiated by configurations.
        """

        rank = [] # the keep in track the configuration and the corresponding performance

        # we save the best trained model and the training loss during the epochs
        best, loss = None, None
        best_dev_acc = 0

        for idx, config in enumerate(configs):
            print("Config: ",idx)

            trainer = sMNISTEsnClassifier(1, hidden_dim=config["units"],
                                          omega=config["omega"],
                                          spectral_radius=config["radius"],
                                          leakage_rate=config["leakage"],
                                          tikhonov=config["tikhonov"],
                                          device=gpu)

            acc_tr = trainer.fit(*tr)
            acc_vl, _  = trainer.predict(*dev)

            rank.append((config, round(acc_tr, 4), round(acc_vl, 4)))

            print(f'Results: Acc tr: {round(acc_tr, 4)}', f'Acc vl: {round(acc_vl, 4)}')

            # we keep the best model
            if best_dev_acc < acc_vl:
                best_dev_acc = acc_vl
                best = copy.deepcopy(trainer)

        return rank, best

## Retrieve the dataset and Hold out

In [4]:
tr_dataset = Sequential_mnist("train", root=".\..\..\Sources\MNIST", one_hot_encoding=True)
dev_dataset = Sequential_mnist("dev", root=".\..\..\Sources\MNIST", one_hot_encoding=True)
ts_dataset = Sequential_mnist("test", root=".\..\..\Sources\MNIST", one_hot_encoding=True)

tr_dataset = (tr_dataset.data.transpose_(0,1).to(gpu), tr_dataset.target.float().to(gpu))
dev_dataset = (dev_dataset.data.transpose_(0,1).to(gpu), dev_dataset.target.float().to(gpu))
ts_dataset = (ts_dataset.data.transpose_(0,1).to(gpu), ts_dataset.target.float().to(gpu))

## Grid search of Classifier based on Echo State Network

In [5]:
ranges_to_explore = {
    "units" : [200, 300, 400],
    "omega" : [0.8, 1],
    "radius" : [1.5, 2, 5, 7, 9],
    "leakage": [0.01, 0.03],
    "tikhonov" : [1e-06]
}

In [6]:
gs = GridSearch(ranges_to_explore, tr_dataset, dev_dataset)
best_config =  gs.best_config
best_model = gs.best_model

Number of configurations to try:  60
Config:  0
Results: Acc tr: 0.4774 Acc vl: 0.485
Config:  1
Results: Acc tr: 0.5333 Acc vl: 0.5524
Config:  2
Results: Acc tr: 0.4865 Acc vl: 0.4994
Config:  3
Results: Acc tr: 0.5443 Acc vl: 0.5678
Config:  4
Results: Acc tr: 0.5757 Acc vl: 0.5986
Config:  5
Results: Acc tr: 0.4055 Acc vl: 0.4125
Config:  6
Results: Acc tr: 0.5841 Acc vl: 0.6047
Config:  7
Results: Acc tr: 0.2997 Acc vl: 0.2906
Config:  8
Results: Acc tr: 0.5741 Acc vl: 0.599
Config:  9
Results: Acc tr: 0.3037 Acc vl: 0.301
Config:  10
Results: Acc tr: 0.3682 Acc vl: 0.3738
Config:  11
Results: Acc tr: 0.3827 Acc vl: 0.3926
Config:  12
Results: Acc tr: 0.4717 Acc vl: 0.4831
Config:  13
Results: Acc tr: 0.6117 Acc vl: 0.6377
Config:  14
Results: Acc tr: 0.6095 Acc vl: 0.6198
Config:  15
Results: Acc tr: 0.5121 Acc vl: 0.5273
Config:  16
Results: Acc tr: 0.5875 Acc vl: 0.6052
Config:  17
Results: Acc tr: 0.3149 Acc vl: 0.3155
Config:  18
Results: Acc tr: 0.5065 Acc vl: 0.5135
Config:

In [7]:
tr_acc, _ = best_model.predict(*tr_dataset)
print(f'Accuracy: {round(tr_acc, 4)}')

dev_acc, _ = best_model.predict(*dev_dataset)
print(f'Accuracy: {round(dev_acc, 4)}')

test_acc, _ = best_model.predict(*ts_dataset)
print(f'Accuracy: {round(test_acc, 4)}')

Accuracy: 0.6567
Accuracy: 0.6797
Accuracy: 0.661


### Final retrain with Training and Validation set (with the best configuration)

In [8]:
final_model = sMNISTEsnClassifier(1, hidden_dim=best_config["units"],
                                  omega=best_config["omega"],
                                  spectral_radius=best_config["radius"],
                                  leakage_rate=best_config["leakage"],
                                  tikhonov=best_config["tikhonov"],
                                  device=gpu)

final_tr = Sequential_mnist("train-dev", root=".\..\..\Sources\MNIST", one_hot_encoding=True)
final_tr = (final_tr.data.transpose_(0,1).to(gpu), final_tr.target.float().to(gpu))

tr_acc = final_model.fit(*final_tr)

print(f'Accuracy: {round(tr_acc, 4)}')

test_acc, _  = final_model.predict(*ts_dataset)
print(f'Accuracy: {round(test_acc, 4)}')

Accuracy: 0.5555
Accuracy: 0.555
